## Comparing Long Term Investment Strategies
### Karandeep Singh

Investments are critical to long term financial security. Through the power of compound interest, investors can grow their wealth over time. However, there are many different investment strategies that can be used to grow wealth.

In this analysis, we will compare the benefits of investing in:
- Cryptocurrency
- Equities in the Largest Economy (US)
- Local Equities (Singapore)
- Emerging Market Equities (India)
- Safe Assets (Gold)
- Low Risk Assets (US Treasury Bonds)
- Professional Management (Mutual Funds)

**The goal of this analysis is to determine which investment strategies are the most effective in growing wealth over time.**

Insights I hope to gain from this analysis include:

1. What benefits does investing offer, compared to holding cash?
2. Which investment vehicles are the most effective in growing wealth over time?
3. Which investment vehicles offer the best risk-adjusted returns?
4. Which investment vehicles are the most volatile?
5. Are developing countries likely to outperform developed countries in the long run?
6. Does GDP growth translate to stock market growth?

This analysis makes use of the following datasets:
- [SPDR S&P 500 ETF Trust (SPY) Historical Prices](https://finance.yahoo.com/quote/SPY/history?period1=1659308442&period2=1690844442&interval=1mo&filter=history&frequency=1mo&includeAdjustedClose=true)
- [Bitcoin USD (BTC-USD) Historical Prices](https://finance.yahoo.com/quote/BTC-USD/history?period1=1659322815&period2=1690858815&interval=1mo&filter=history&frequency=1mo&includeAdjustedClose=true)
- [FTSE Straits Times Index STI Historical Prices](https://www.wsj.com/market-data/quotes/index/SG/STI/historical-prices)
- ADD NIFTY HERE
- [Gold Spot Prices](https://www.gold.org/goldhub/data/gold-prices)
- [U.S. Department of the Treasury - Interest Rates Data CSV Archive](https://home.treasury.gov/interest-rates-data-csv-archive)
- [Fidelity Growth Company Fund Historical Quotes](https://www.marketwatch.com/investing/fund/fdgrx/download-data)
- [Singapore Department of Statistics - Consumer Price Index (CPI), 2019 As Base Year](https://tablebuilder.singstat.gov.sg/table/TS/M212881)
- [U.S. Department of Labor - Consumer Price Index (CPI), 1982-1984 As Base Year](https://www.bls.gov/cpi/data.htm)

In [1]:
# Importing required libraries
import pandas as pd
import numpy as np
import seaborn as sns

### Loading and Preprocessing the Datasets

The ***SPDR S&P 500 ETF Trust (SPY) Historical Prices*** dataset contains the historical prices of the SPY ETF. This is a fund that tracks the performance of the 500 largest companies in the USA. It is the most commonly used proxy for the entire US stock market.

We can discard all columns apart from the date and the adjusted close, which is the price of the ETF at the end of each trading day and adjusted for dividends and stock splits. A stock split is a process where a single share is split into multiple shares, which reduces the price of each share and can cause our data to be incorrect unless we adjust for it.

In [2]:
spy_data = pd.read_csv('data/SPY.csv')
spy_data = spy_data[['Date', 'Adj Close']]
# Date is formatted as yyyy-mm-dd, convert to datetime
spy_data['Date'] = pd.to_datetime(spy_data['Date'])
spy_data.head()

,Date,Adj Close
0,1993-01-29,25.029373
1,1993-02-01,25.207396
2,1993-02-02,25.260807
3,1993-02-03,25.527821
4,1993-02-04,25.634634


The ***Bitcoin USD (BTC-USD) Historical Data*** dataset contains historical data of Bitcoin prices from 2014 to 2021. Bitcoin is one of the oldest and most popular cryptocurrencies in the world, making it a good choice for this analysis. Unlike most other cryptocurrencies and cryptocurrency brokers, Bitcoin does not appear to be a scam, despite the fact that its only real value is in purchasing drugs, weapons, politicians, and other deplorable things on the dark web.

As with the SPY dataset, we're only concerned with the date and adjusted close columns.

In [3]:
btc_data = pd.read_csv('data/BTC-USD.csv')
btc_data = btc_data[['Date', 'Adj Close']]
# Date is formatted as yyyy-mm-dd, convert to datetime
btc_data['Date'] = pd.to_datetime(btc_data['Date'])
btc_data.head()

,Date,Adj Close
0,2014-09-17,457.334015
1,2014-09-18,424.440002
2,2014-09-19,394.795990
3,2014-09-20,408.903992
4,2014-09-21,398.821014


The ***FTSE Straits Times Index STI Historical Prices*** dataset contains the historical prices of the Straits Times Index (STI) from 2000 to 2019. Like SPY, STI is an index that can be used as a proxy for the Singapore stock market. It tracks the 30 largest companies listed on the Singapore Exchange (SGX).

STI has never had a stock split, so the price data does not need to be adjusted.

In [4]:
sti_data = pd.read_csv('data/STI.csv')
sti_data = sti_data[['Date', ' Close']]
# Date is formatted as mm/dd/yy, convert to datetime
sti_data['Date'] = pd.to_datetime(sti_data['Date'], format='%m/%d/%y')
sti_data.head()

,Date,Close
0,2023-07-31,3373.98
1,2023-07-28,3371.17
2,2023-07-27,3337.42
3,2023-07-26,3304.96
4,2023-07-25,3286.16


The ***Gold Spot Prices*** dataset records gold prices (per ounce) from 1978 to 2023. Unlike most of the other datasets, it is provided as a Microsoft Excel workbook with multiple sheets.

For maximum precision and granularity, we will use the "Daily" sheet, as this contains daily prices. The first 5 rows contain metadata, so we will skip them. The first 2 columns are empty and purely for formatting, so we will skip them as well. The remaining columns contain the Date, and the prices in various currencies. We will only use the USD column for a more consistent comparison with the other datasets.

In [5]:
gold_xlsx_data = pd.read_excel('data/GoldSpotPrices.xlsx', sheet_name='Daily', skiprows=5, index_col=2)
gold_xlsx_data = gold_xlsx_data.iloc[:, [0, 2]]
gold_xlsx_data.head()

,Unnamed: 0,USD
1978-12-29,NaN,226.00
1979-01-01,NaN,226.00
1979-01-02,NaN,226.80
1979-01-03,NaN,218.60
1979-01-04,NaN,223.15
